In [208]:
#Import Python Libraries
import numpy as np
import pandas as pd
import tweepy
import csv
import re
import string 

In [209]:
#Read csv file
df = pd.read_csv('newExtractedData (1) (1) (1).csv')

In [210]:
df.dtypes

date          object
username      object
to            object
replies        int64
retweets       int64
favorites      int64
text          object
geo          float64
mentions      object
hashtags      object
id           float64
permalink     object
stance        object
dtype: object

In [211]:
df.head(5)

,date,username,to,replies,retweets,favorites,text,geo,mentions,hashtags,id,permalink,stance
0,07/03/2020,XHrb9,AlRajhiBankCare,1,0,0,السلام عليكم هل اقدر ابيع جزء من اسهم اكتتاب ا...,NaN,NaN,NaN,1.236390e+18,https://twitter.com/XHrb9/status/1236391193879...,Neutral
1,07/03/2020,metrekmeshal,NaN,17,3,6,#تاسي مساء الخير ... غداً تستأنف التداولات في ...,NaN,NaN,NaN,1.236240e+18,https://twitter.com/metrekmeshal/status/123623...,Neutral
2,07/03/2020,Nawaf44930655,vip9tasi,0,0,0,والله يابو دانة لعنة انهيار السوق ٢٠٠٦ حتى الي...,NaN,NaN,NaN,1.236230e+18,https://twitter.com/Nawaf44930655/status/12362...,Negative
3,07/03/2020,A5O_FAZZAH,DrAbdullahGh_,0,0,0,استثمار مادي في العلاقه ! هذا اكتتاب ارامكو مو حب,NaN,NaN,NaN,1.236230e+18,https://twitter.com/A5O_FAZZAH/status/12362277...,Neutral
4,07/03/2020,hamimi50,alwardit,0,0,1,بالنسبة للميزانية السعوديه فقد خرجت من أزمة اس...,NaN,NaN,NaN,1.236200e+18,https://twitter.com/hamimi50/status/1236201906...,Positive


In [212]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [213]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

In [223]:
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [224]:
#start processing the tweet
def processPost(tweet):

    #Replace @username with empty string
    tweet = re.sub('@[^\s]+', ' ', str(tweet))
    
    #Convert www.* or https?://* to " "
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',str(tweet))
    
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)

    # remove punctuations
    tweet= remove_punctuations(str(tweet))
    
    # normalize the tweet
    tweet= normalize_arabic(str(tweet))
    
    # remove repeated letters
    tweet=remove_repeating_char(tweet)

    #Remove new lines with white space
    tweet = str(tweet).replace('\n',' ')
    tweet = str(tweet).replace('\t',' ')
        
    return tweet

In [225]:
# remove Arabic stop words from the tweets
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
ar_stops = set(stopwords.words('arabic'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [226]:
# remove Arabic stop words from the tweets
for word in df['text']:
    if word in ar_stops:        
        words_filtered.remove(word)

In [227]:
df["clean text"] = df['text'].apply(lambda x: processPost(x))

In [228]:
df.tail(5)

,date,username,to,replies,retweets,favorites,text,geo,mentions,hashtags,id,permalink,stance,clean text
5723,05/12/2019,AlamlRoooo7,NaN,0,0,1,#اكتتاب_أرامكو هو الأكبر بالعالم • إجمالي الأم...,NaN,NaN,NaN,1.202710e+18,https://twitter.com/AlamlRoooo7/status/1202707...,positive,اكتابارامكو هو الاكبر بالعالم • اجمالي الاموال...
5724,05/12/2019,KBri32457754,NaN,0,0,1,#اكتتاب_أرامكو يصبح الأكبر في العالم بقيمة 25....,NaN,NaN,NaN,1.202710e+18,https://twitter.com/KBri32457754/status/120270...,positive,اكتابارامكو يصبح الاكبر في العالم بقيمه 256 مل...
5725,05/12/2019,0123Bin,NaN,1,1,1,ارتفاع عدد المكتتبين الأفراد في #أرامكو لأكثر ...,NaN,NaN,NaN,1.202710e+18,https://twitter.com/0123Bin/status/12027077519...,Neutral,ارتفاع عد المكتبين الافراد في ارامكو لاكثر من ...
5726,05/12/2019,Esam7736,NaN,0,0,0,#اكتتاب_ارامكو بما اني احد ملاك اسهم في شركة ا...,NaN,NaN,NaN,1.202710e+18,https://twitter.com/Esam7736/status/1202707750...,Neutral,اكتابارامكو بما اني احد ملاك اسهم في شركه ارام...
5727,05/12/2019,alqabas,NaN,1,4,2,NaN,NaN,NaN,NaN,1.202710e+18,https://twitter.com/alqabas/status/12027077163...,positive,nan
